# Similarity Index

In [ ]:
# function for SSIM and MSE and MS-SSIM

from skimage.metrics import structural_similarity as ssim
from skimage.metrics import structural_similarity as compare_ssim
from skimage import metrics
from skimage import measure
def mse(imageA, imageB):
    # the 'Mean Squared Error' between the two images is the
    # sum of the squared difference between the two images;
    # NOTE: the two images must have the same dimension
    err = np.sum((imageA.astype("float") - imageB.astype("float")) ** 2)
    err /= float(imageA.shape[0] * imageA.shape[1])# return the MSE, the lower the error, the more "similar"
    # the two images are
    return err
def compare_images(imageA, imageB):
    # compute the mean squared error and structural similarity
    # index for the images
    s = ssim(imageA, imageB,multichannel=True)
    return s
def multissim(imageA, imageB):
    ms_ssim = compare_ssim(imageA, imageB,multichannel=True)
    return ms_ssim

def euclidian_dis(imageA, imageB):
    # Ensure both images have the same shape (e.g., resize them)
    imageA = np.array(imageA)
    imageB = np.array(imageB)
    if imageA.shape != imageB.shape:
        common_shape = (min(imageA.shape[0], imageB.shape[0]), min(imageA.shape[1], imageB.shape[1]))
        imageA = imageA[:common_shape[0], :common_shape[1]]
        imageB = imageB[:common_shape[0], :common_shape[1]]

    # Calculate the Euclidean distance between the two images
    euclidean_distance = np.linalg.norm(imageA - imageB)
    return euclidean_distance




In [ ]:
import pandas as pd
def compare_images_in_directory(directory_path, target_image_path):
    target_image = cv.imread(target_image_path)
    if target_image is None:
        print(f"Failed to read the target image: {target_image_path}")
        return

    data = {
        'Image Name': [],
        'SSIM Score': [],
        'MSE Error': [],
        'MS-SSIM Score': [],
        'Euclidean Distance': []
    }

    for filename in os.listdir(directory_path):
        if filename.lower().endswith((".jpg", ".jpeg", ".png")):
            image_path = os.path.join(directory_path, filename)
            image = cv.imread(image_path)

            if target_image.shape != image.shape:
                a, b, c = target_image.shape
                image = cv.resize(image, dsize=(b, a), interpolation=cv.INTER_LINEAR)

                if image is not None:
                    ssim_score = compare_images(target_image, image)
                    mse_error=mse(target_image, image)
                    ms_ssim_score=multissim(target_image, image)
                    eucl_score=euclidian_dis(target_image, image)

                    data['Image Name'].append(filename)
                    data['SSIM Score'].append(ssim_score)
                    data['MSE Error'].append(mse_error)
                    data['MS-SSIM Score'].append(ms_ssim_score)
                    data['Euclidean Distance'].append(eucl_score)

    df = pd.DataFrame(data)
    df.to_csv('/content/image_comparison_results_2.csv', index=False)




In [ ]:
# CSV FILE 1
#Example usage:
directory_path = 'image directory in which we extracted the features'
target_image_path = 'target image of the person whose picture was used for deepfake ( try to use original image which was used to swapp the video)'
compare_images_in_directory(directory_path, target_image_path)

In [ ]:
def orb(img1,img2):


In [ ]:
# CSV FILE 2

import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import ContrastiveLoss
from tensorflow.keras.metrics import BinaryAccuracy

import pandas as pd
# Function to create a simple Convolutional Neural Network (CNN)
def create_cnn_model(input_shape):
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(32, activation='relu'))
    return model

# Load images from a directory
def load_images_from_directory(directory):
    images = []
    filenames = []
    for filename in os.listdir(directory):
        if filename.endswith(".jpg") or filename.endswith(".png"):
            img_path = os.path.join(directory, filename)
            img = cv2.imread(img_path)
            img = cv2.resize(img, (64, 64))  # Resize images to a consistent size
            images.append(img)
            filenames.append(filename)
    return np.array(images), filenames

# Define the input shape based on your image size and channels
input_shape = (64, 64, 3)

# Specify the directory containing your images
image_directory = "/content/drive/MyDrive/celeb_face_ext"

# Load images and filenames from the directory
image_data, filenames = load_images_from_directory(image_directory)

# Add batch dimension and normalize the images
image_data = image_data / 255.0  # Normalize pixel values to the range [0, 1]

# Create and compile the CNN model
model = create_cnn_model(input_shape)
model.compile(optimizer='adam', loss='mse')

# Extract features from the images using the CNN
features = model.predict(image_data)

# Load the target image
target_image_path = "/content/deepfakend data.jpg"
target_image = cv2.imread(target_image_path)
target_image = cv2.resize(target_image, (64, 64))  # Resize to match other images
target_image = target_image / 255.0  # Normalize pixel values to the range [0, 1]
target_image = np.expand_dims(target_image, axis=0)  # Add batch dimension

# Extract features from the target image using the CNN
target_features = model.predict(target_image)

# Calculate Euclidean distances between the target image and all other images
distances = []

for i in range(features.shape[0]):
    distance = euclidean(target_features.flatten(), features[i].flatten())
    distances.append(distance)

# Create a DataFrame with filenames and distances
result_df = pd.DataFrame({'Filename': filenames, 'Euclidean Distance': distances})

# Save the DataFrame to a CSV file
result_df.to_csv('/content/drive/MyDrive/conv_euc_results.csv', index=False)

# Print the DataFrame
print(result_df)


In [ ]:
#  TO JOIN THE TWO DIFFERNT CSV FILES WE GET
#import pandas as pd
df1 = pd.read_csv('/content/image_comparison_results(4).csv')
df2 = pd.read_csv('/content/image_comparison_results_2.csv')

# Concatenate along rows (axis=0)
result = pd.concat([df1, df2], axis=0, ignore_index=True)
# Assuming 'result' is the concatenated DataFrame
result.to_csv('/content/concatenated_celeb_data.csv', index=False)

# Print the concatenated result
print(result)